In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [2]:
len(symbols)

178

## MB-iSTFT-VITS

In [3]:
hps = utils.get_hparams_from_file("./configs/ljs_ms_istft_vits.json")

In [4]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("./checkpoints/pretrained_ms_istft_vits_ddp.pth", net_g, None)

Mutli-stream iSTFT VITS


In [8]:
import time

b = time.time()
stn_tst = get_text("ONNX Runtime supports all opsets from the latest released version of the ONNX spec. github is a opensource website ", hps)
print(f'{stn_tst.shape = }', time.time() - b)
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    print(f'{x_tst.shape = }')
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    print(f'{x_tst_lengths.shape = }')
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=0.9, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()

print(time.time() - b)
#ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

stn_tst.shape = torch.Size([257]) 0.5409104824066162
x_tst.shape = torch.Size([1, 257])
x_tst_lengths.shape = torch.Size([1])
2.2662770748138428
